In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("jcprogjava/handwritten-digits-dataset-not-in-mnist")

print("Path to dataset files:", path)
# Install necessary libraries
!pip install opencv-python pandas scikit-learn joblib tensorflow

# Import required libraries
import cv2
import csv
import os
import numpy as np
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn import metrics
import joblib
from google.colab import files
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

# Step 1: Upload digit images
def upload_images():
    print("Upload images of handwritten digits. Use folder names as labels (e.g., 0, 1, 2, ...).")
    uploaded = files.upload()

    os.makedirs('captured_images', exist_ok=True)

    # Create directories for each label (0-9) if they don't exist
    for i in range(10):
        os.makedirs(f"captured_images/{i}", exist_ok=True)

    # Move uploaded images into corresponding folders
    for filename in uploaded.keys():
        digit_label = input(f"Enter the digit label for {filename}: ")
        if digit_label.isdigit() and int(digit_label) in range(10):
            destination = f"captured_images/{digit_label}/{filename}"
            with open(destination, "wb") as f:
                f.write(uploaded[filename])
            print(f"Image {filename} saved to folder {digit_label}")
        else:
            print(f"Invalid label for {filename}. Please upload images labeled from 0 to 9.")

    print("Images uploaded and saved to 'captured_images'.")

# Step 2: Generate Dataset from Uploaded Images
def generate_dataset():
    header = ["label"] + [f"pixel{i}" for i in range(784)]
    with open('dataset.csv', 'w') as f:
        writer = csv.writer(f)
        writer.writerow(header)

    for label in os.listdir('captured_images'):
        folder_path = os.path.join('captured_images', label)
        if not os.path.isdir(folder_path):
            continue
        for img_file in os.listdir(folder_path):
            img_path = os.path.join(folder_path, img_file)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (28, 28), interpolation=cv2.INTER_AREA)

            # Apply additional preprocessing
            img = cv2.equalizeHist(img)  # Improve contrast
            img = cv2.GaussianBlur(img, (3, 3), 0)  # Reduce noise

            data = [int(label)] + img.flatten().tolist()
            with open('dataset.csv', 'a') as f:
                writer = csv.writer(f)
                writer.writerow(data)
    print("Dataset created and saved as 'dataset.csv'.")

# Step 3: Train Model and Save It (Optimized for Small Datasets)
def train_model_cnn():
    # Load and preprocess data
    data = pd.read_csv('dataset.csv')
    data = shuffle(data)
    X = data.drop("label", axis=1)
    y = data["label"]

    # Check number of samples per class
    class_counts = y.value_counts()
    print("Samples per digit:", class_counts)

    # Check if you have enough samples
    if len(X) < 10:
        print("ERROR: Insufficient training data!")
        print("Please upload more images for each digit class.")
        return

    # Reshape and normalize
    X = X.values.reshape(-1, 28, 28, 1).astype('float32') / 255.0
    y = tf.keras.utils.to_categorical(y, 10)  # One-hot encoding labels

    # Modify train-test split for small datasets
    if len(X) < 20:
        # If very few samples, use a minimal split or leave-one-out approach
        train_x, test_x = X[:-1], X[-1:]
        train_y, test_y = y[:-1], y[-1:]
    else:
        # Use standard split if enough samples
        train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.2)

    # Simplified CNN architecture for small datasets
    model = models.Sequential([
        layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1), padding='same'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(10, activation='softmax')
    ])

    # Compile the model
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    # Aggressive Data Augmentation for Small Datasets
    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=False,
        fill_mode='nearest'
    )

    # Fit the generator on the training data
    datagen.fit(train_x)

    # Train with more epochs and data augmentation
    history = model.fit(
        datagen.flow(train_x, train_y, batch_size=len(train_x)),
        epochs=100,  # Increased epochs for small datasets
        validation_data=(test_x, test_y)
    )

    # Evaluate the model
    if len(test_x) > 0:
        test_loss, test_accuracy = model.evaluate(test_x, test_y)
        print(f"Test accuracy: {test_accuracy * 100:.2f}%")

    # Save the trained model
    model.save('model/digit_recognizer_small_dataset_cnn.h5')
    print("CNN Model trained and saved.")

# Step 4: Predict Digits
def predict_digit():
    print("Upload an image of a handwritten digit for prediction.")
    uploaded = files.upload()

    try:
        model = tf.keras.models.load_model('model/digit_recognizer_small_dataset_cnn.h5')
    except:
        print("No trained model found. Please train the model first.")
        return

    for filename in uploaded.keys():
        # Read image and preprocess
        img = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (28, 28), interpolation=cv2.INTER_AREA)
        img = cv2.equalizeHist(img)  # Improve contrast
        img = cv2.GaussianBlur(img, (3, 3), 0)  # Reduce noise

        # Normalize and reshape
        img = img.reshape(1, 28, 28, 1).astype('float32') / 255.0

        # Predict
        prediction = model.predict(img)
        predicted_label = np.argmax(prediction, axis=1)
        confidence = np.max(prediction) * 100
        print(f"Predicted digit: {predicted_label[0]}")
        print(f"Confidence: {confidence:.2f}%")

In [ ]:
# Main Execution loop to handle repeated choices
def main():
    while True:
        print("\nOptions:\n1. Upload Images\n2. Generate Dataset\n3. Train Model\n4. Predict a Digit\n5. Exit")
        choice = input("Enter your choice: ")
        if choice == "1":
            upload_images()
        elif choice == "2":
            generate_dataset()
        elif choice == "3":
            train_model_cnn()
        elif choice == "4":
            predict_digit()
        elif choice == "5":
            print("Exiting the program.")
            break
        else:
            print("Invalid choice. Please try again.")

# Run the main loop
main()


Options:
1. Upload Images
2. Generate Dataset
3. Train Model
4. Predict a Digit
5. Exit
Upload images of handwritten digits. Use folder names as labels (e.g., 0, 1, 2, ...).


Saving 8.png to 8.png
Saving 9 - Copy.png to 9 - Copy.png
Saving 9.png to 9.png
Saving 6.png to 6.png
Saving 1 - Copy.png to 1 - Copy.png
Saving 1.png to 1.png
Saving 5.png to 5.png
Image 8.png saved to folder 8
Image 9 - Copy.png saved to folder 9
Image 9.png saved to folder 9
Image 6.png saved to folder 6
Image 1 - Copy.png saved to folder 1
Image 1.png saved to folder 1
Image 5.png saved to folder 5
Images uploaded and saved to 'captured_images'.

Options:
1. Upload Images
2. Generate Dataset
3. Train Model
4. Predict a Digit
5. Exit
Dataset created and saved as 'dataset.csv'.

Options:
1. Upload Images
2. Generate Dataset
3. Train Model
4. Predict a Digit
5. Exit
Samples per digit: label
9    2
1    2
5    1
6    1
8    1
Name: count, dtype: int64
ERROR: Insufficient training data!
Please upload more images for each digit class.

Options:
1. Upload Images
2. Generate Dataset
3. Train Model
4. Predict a Digit
5. Exit
Upload images of handwritten digits. Use folder names as labels (e